# Getting the most solar power for your dollar
## Data wrangling
### Zachary Brown

The goal of this project is to use the Tracking the Sun dataset from the Lawrence Berkeley National Laboratory to create a model that identifies what factors make residential solar panel installations in Austin, Texas as cost-efficient as possible. To do so the data will be loaded from Parquet files and in this notebook will be wrangled into the working dataframe for the rest of the project.

I'll start off by importing the modules needed for importing and wrangling the data.

In [8]:
import os
import wget
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from datetime import datetime

Next I need to download the Parquet datafiles into the external data directory within this repository, so I'll switch to the correct directory.

In [5]:
print(os.getcwd())
os.chdir(r"..\data\external")
data_dir = os.getcwd()
print(os.getcwd())

C:\Users\Zachary Brown\Documents\GitHub\Solar-Panel-Capstone\notebooks
C:\Users\Zachary Brown\Documents\GitHub\Solar-Panel-Capstone\data\external


The next step is to download the parquet files. The entire collection of data is hosted here: https://data.openei.org/s3_viewer?bucket=oedi-data-lake&prefix=tracking-the-sun%2F

I only need 2021 data for this project as I want the most up to date model possible. Within 2021 the data is broken down by state. I need to download all 26 files, so I'll create a function to automatically download them all for me.

In [9]:
# Start by creating the function and creating empty url and file name lists

def import_data(root_url):
    urls = []
    file_names = []
    
# Parse the 2021 data directory to get URLs for each state    

    response = requests.get(root_url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    for a in soup.find_all('a', href=True):
        if '/s3_viewer?bucket=oedi-data-lake&prefix=tracking-the-sun%2F' in a['href']:
            urls.append('https://data.openei.org' + a['href'])
    
# Create a list of the file names for each state from each link in the urls list

    for url in urls:
        url_response = requests.get(url)
        url_soup = BeautifulSoup(url_response.content, 'html.parser')
        
        for a in url_soup.find_all('a', href=True):
            if '.parquet' in a['href']:
                file_names.append(a['href'])
            
# Loop through each file and download to data directory

    for url, file in zip(urls, file_names):
        wget.download(file)

Now that the function has been created to download all of the Parquet files locally we need to run it with the 2021 URL to download the state data files.

In [10]:
import_data('https://data.openei.org/s3_viewer?bucket=oedi-data-lake&prefix=tracking-the-sun%2F2021%2F')

Now the data files need to be loaded into a single Pandas dataframe to begin the wrangling process.

In [30]:
file_list = []

# I'll start by adding each Parquet file to an empty list.
for file in os.listdir(data_dir):
    if file.endswith('.parquet'):
        file_list.append(file)
        
# Now I'll iterate over the file_list and read each in to one dataframe, aggregating all of the data.
tables = []

for filename in file_list:
    df = pd.read_parquet(filename)
    tables.append(df)

raw_data = pd.concat(tables, ignore_index=True)
raw_data.head()

,data_provider_1,data_provider_2,system_id_1,system_id_2,installation_date,system_size_dc,total_installed_price,rebate_or_grant,customer_segment,expansion_system,...,output_capacity_inverter_1,output_capacity_inverter_2,output_capacity_inverter_3,dc_optimizer,inverter_loading_ratio,date_of_battery_install,battery_manufacturer,battery_model,battery_rated_capacity_kw,battery_rated_capacity_kwh
0,"Washington, D.C. Public Service Commission",-1,DC-2012700-SUN-I,-1,2020-11-30,6.80,-1.0,-1.0,-1,0,...,-1.0,-1.0,-1.0,-1,-1.0,NaT,-1,-1,-1.0,-1.0
1,"Washington, D.C. Public Service Commission",-1,DC-2012701-SUN-I,-1,2020-12-07,7.04,-1.0,-1.0,-1,0,...,-1.0,-1.0,-1.0,-1,-1.0,NaT,-1,-1,-1.0,-1.0
2,"Washington, D.C. Public Service Commission",-1,DC-2012702-SUN-I,-1,2020-12-07,7.68,-1.0,-1.0,-1,0,...,-1.0,-1.0,-1.0,-1,-1.0,NaT,-1,-1,-1.0,-1.0
3,"Washington, D.C. Public Service Commission",-1,DC-2012703-SUN-I,-1,2020-12-06,5.12,-1.0,-1.0,-1,0,...,-1.0,-1.0,-1.0,-1,-1.0,NaT,-1,-1,-1.0,-1.0
4,"Washington, D.C. Public Service Commission",-1,DC-2012704-SUN-I,-1,2020-12-07,5.76,-1.0,-1.0,-1,0,...,-1.0,-1.0,-1.0,-1,-1.0,NaT,-1,-1,-1.0,-1.0


Ok, now that the dataframe has been assembled it's time to explore it.

In [26]:
raw_data.shape

(2041551, 80)

In [14]:
raw_data.columns

Index(['data_provider_1', 'data_provider_2', 'system_id_1', 'system_id_2',
       'installation_date', 'system_size_dc', 'total_installed_price',
       'rebate_or_grant', 'customer_segment', 'expansion_system',
       'multiple_phase_system', 'new_construction', 'tracking',
       'ground_mounted', 'zip_code', 'city', 'utility_service_territory',
       'third_party_owned', 'installer_name', 'self_installed', 'azimuth_1',
       'azimuth_2', 'azimuth_3', 'tilt_1', 'tilt_2', 'tilt_3',
       'module_manufacturer_1', 'module_model_1', 'module_quantity_1',
       'module_manufacturer_2', 'module_model_2', 'module_quantity_2',
       'module_manufacturer_3', 'module_model_3', 'module_quantity_3',
       'additional_modules', 'technology_module_1', 'technology_module_2',
       'technology_module_3', 'bipv_module_1', 'bipv_module_2',
       'bipv_module_3', 'bifacial_module_1', 'bifacial_module_2',
       'bifacial_module_3', 'nameplate_capacity_module_1',
       'nameplate_capacity_module

In [15]:
col = raw_data.columns.to_series().groupby(df.dtypes).groups
print(col)

{datetime64[ns]: ['installation_date', 'date_of_battery_install'], int32: ['expansion_system', 'multiple_phase_system', 'new_construction', 'tracking', 'ground_mounted', 'third_party_owned', 'self_installed', 'module_quantity_1', 'module_quantity_2', 'module_quantity_3', 'additional_modules', 'bipv_module_1', 'bipv_module_2', 'bipv_module_3', 'bifacial_module_1', 'bifacial_module_2', 'bifacial_module_3', 'nameplate_capacity_module_1', 'nameplate_capacity_module_2', 'nameplate_capacity_module_3', 'inverter_quantity_2', 'inverter_quantity_3', 'additional_inverters', 'micro_inverter_1', 'micro_inverter_2', 'micro_inverter_3', 'solar_storage_hybrid_inverter_1', 'solar_storage_hybrid_inverter_2', 'solar_storage_hybrid_inverter_3', 'built_in_meter_inverter_1', 'built_in_meter_inverter_2', 'built_in_meter_inverter_3', 'output_capacity_inverter_2', 'output_capacity_inverter_3', 'dc_optimizer'], float64: ['system_size_dc', 'total_installed_price', 'rebate_or_grant', 'azimuth_1', 'azimuth_2', 'a

In [28]:
long_zip = len(raw_data['zip_code']) > 5
type(long_zip)
print(long_zip)

True


Right now I'm missing a state column, and although zip code is object type, it includes 9 digit zip codes with hyphens which isn't a problem.

Before I create a state column I'll pare down the data to only applicable rows. This means just residential installations from 2021 that are not missing 'total_installed_price' values

In [34]:
print(raw_data['customer_segment'].unique())

['-1' 'RES' 'NON-PROFIT' 'SCHOOL' 'GOV' 'NON-RES' 'COM']


In [35]:
# Now I'll subset the data to just residential installations
res_data = raw_data[raw_data['customer_segment'] == 'RES']
res_data.shape

(1948072, 80)

In [60]:
res_copy = res_data.copy()
res_copy['year'] = res_copy.installation_date.dt.year
res_copy['year'].value_counts().sort_index()

1971.0         1
1985.0         3
1986.0         6
1988.0         5
1989.0         5
1990.0         6
1991.0        10
1992.0         8
1993.0         4
1994.0         7
1995.0        10
1996.0         8
1997.0        12
1998.0        32
1999.0       170
2000.0       175
2001.0      1139
2002.0      2181
2003.0      2818
2004.0      4956
2005.0      5052
2006.0      8309
2007.0     12497
2008.0     14394
2009.0     26267
2010.0     36691
2011.0     47583
2012.0     66465
2013.0    103238
2014.0    154819
2015.0    245536
2016.0    278337
2017.0    218597
2018.0    223471
2019.0    240816
2020.0    238227
2021.0      2266
2106.0         8
Name: year, dtype: int64

Interestingly it looks like there's not actually very much 2021 data in the Parquet files, so I'll use 2020 and 2021 data together to build the model for this project. 

In [67]:
newest_data = res_copy[res_copy['installation_date'].dt.year.isin([2020, 2021])]
newest_data.shape

(240493, 81)

Next I want to check how many rows are missing 'total_installed_price' data and remove them since that price is key to calculating the final installed price, which is the key metric for the model. Missing values are indicated as -1 in this dataset.

In [72]:
newest_data['total_installed_price'].value_counts().sort_index()

0.01             8
0.10             1
1.00          1937
3.25             9
3.30             6
              ... 
3680000.00       1
4789800.00       1
7485834.00       1
8255000.00       1
9887777.00       1
Name: total_installed_price, Length: 78724, dtype: int64

In [71]:
missing_sale = newest_data['total_installed_price'] == -1
newest_data = newest_data[~missing_sale]
newest_data.shape

(208464, 81)